# Migrate MYSQL db to PSQL GCP db

In [1]:
import psycopg2
import pandas as pd
import mysql.connector


In [3]:
import db.db_access as access

## set the queries for both db

In [69]:
CONST_SQL_GET_COMP_NAME = 'SELECT * FROM altdata.company;'
CONST_SQL_GET_REP_SALES = 'SELECT * FROM altdata.reported_sales'
CONST_SQL_GET_HIST_PRICES = 'SELECT * FROM altdata.historical_prices'
CONST_SQL_GET_DAILY_ESTIMATION = 'SELECT * FROM altdata.daily_sales_estimation;'

def first_column(array_2d):
    return list(zip(*array_2d))[0]


def db_result_to_pandas(cursor_fetch_result):
    return pd.DataFrame(cursor_fetch_result['result'], columns=cursor_fetch_result['header'])


class MYSQL_connector(object):
    """
    Class for getting data from the database.
    Currently not SQL injection safe.
    TODO: make it safe from SQL injections
    """
    def __init__(self):
        server, database, username, password, driver = access.parameter()

        self.cnx = mysql.connector.connect(user=username, password=password,
                                           host=server,
                                           database=database)

    def get_mysql_context(self):
        server, database, username, password, driver = access.parameter()

        cnx = mysql.connector.connect(user=username, password=password,
                                           host=server,
                                           database=database)
        return cnx

    


In [70]:
GET_COMPANY = 'SELECT * FROM maincompany'



class psql_db_gcp_connector(object):
    def __init__(self):
        self.host, self.port, self.database, self.user, self.password = access.postgre_access_google_cloud()

    def get_psql_context(self):
        cnx = psycopg2.connect(host=self.host, port=self.port, database=self.database, user=self.user, password=self.password)
        return cnx

    def get_company(self):
        cnx = self.get_psql_context()
        cur = cnx.cursor()
        cur.execute(GET_COMPANY)
        result = cur.fetchall()
        result = pd.DataFrame.from_records(result, columns=[x[0] for x in cur.description])
        return result

,company_id,company_name,ticker,active
0,178,1-800-FLOWERS.COM INC-CL A,FLWS US,1
1,503,500.COM LTD-CLASS A-ADR,WBAI US,0
2,2,AARON'S INC,AAN US,1
3,18,ABERCROMBIE & FITCH CO-CL A,ANF US,1
4,28,ACTIVISION BLIZZARD INC,ATVI US,1
...,...,...,...,...
509,529,ZALANDO SE,ZAL GR,1
510,530,ZENDESK INC,ZEN US,1
511,528,ZILLOW GROUP INC - C,Z US,1
512,532,ZOE'S KITCHEN INC,ZOES US,0


In [25]:
psql_company = psql_db.get_company()
psql_company

,id,name,date,ticker,active
0,547,1-800-flowers.com inc-cl a,2020-05-15 13:41:10.350521+00:00,FLWS US,True
1,548,500.com ltd-class a-adr,2020-05-15 13:41:10.350521+00:00,WBAI US,False
2,549,aaron's inc,2020-05-15 13:41:10.350521+00:00,AAN US,True
3,550,abercrombie & fitch co-cl a,2020-05-15 13:41:10.350521+00:00,ANF US,True
4,551,activision blizzard inc,2020-05-15 13:41:10.350521+00:00,ATVI US,True
...,...,...,...,...,...
521,11,Debenhams plc,2019-07-20 02:12:22.801180+00:00,DBNHF US,True
522,2,Marks and Spencers,2019-06-13 12:55:03.325831+00:00,MKS LN,True
523,31,Helen of Troy Limited,2020-04-27 11:55:13.604688+00:00,HELE US,False
524,804,J.C. Penney co inc,2020-05-15 13:41:10.350521+00:00,JCPNQ US,True


## main function that will do the conversion

In [74]:
# function to change the comapny id of the table
def change_company_id(psql_company, mysql_company, id):
    ticker = mysql_company[mysql_company['company_id']==id]['ticker'].values[0]
    
    new_id_ix = list(psql_company['ticker']).index(ticker)
    new_id = psql_company.at[new_id_ix, 'id']
        
    return new_id

## migration of reported sales table

In [55]:
mysql_reported_sales = mysql_db.get_reported_sales_pandas()
mysql_reported_sales = mysql_reported_sales.drop(columns={'id_reported_sales'})
mysql_reported_sales

,company_id,quarter_group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time_announcement
0,1,05q1,2005-03-31,2004-12-31,2005-04-20,4.750000e+09,4.750000e+09,00:00:00
1,1,05q2,2005-06-30,2005-04-01,2005-07-20,5.309000e+09,5.309000e+09,00:00:00
2,1,05q3,2005-09-30,2005-07-01,2005-10-19,5.485000e+09,5.485000e+09,00:00:00
3,1,05q4,2005-12-31,2005-10-01,2006-01-18,5.168000e+09,5.168000e+09,00:00:00
4,1,06q1,2006-03-31,2006-01-01,2006-04-19,5.344000e+09,5.344000e+09,00:00:00
...,...,...,...,...,...,...,...,...
21232,534,19q1,2019-03-31,2019-01-01,2019-05-09,8.558200e+07,8.558200e+07,16:03:00
21233,534,19q2,2019-06-30,2019-04-01,2019-08-08,8.807500e+07,8.807500e+07,16:03:00
21234,534,19q3,2019-09-30,2019-07-01,2019-11-07,9.055500e+07,9.055500e+07,16:03:00
21235,534,19q4,2019-12-31,2019-10-01,2020-02-20,8.966800e+07,8.966800e+07,16:03:00


In [65]:
# change the company_id of the table
psql_reported_sales = mysql_reported_sales.copy()
psql_reported_sales['company_id'] = psql_reported_sales['company_id'].map(lambda x: change_company_id(psql_company, mysql_company, x), list(psql_reported_sales['company_id']))

psql_reported_sales

,company_id,quarter_group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time_announcement
0,562,05q1,2005-03-31,2004-12-31,2005-04-20,4.750000e+09,4.750000e+09,00:00:00
1,562,05q2,2005-06-30,2005-04-01,2005-07-20,5.309000e+09,5.309000e+09,00:00:00
2,562,05q3,2005-09-30,2005-07-01,2005-10-19,5.485000e+09,5.485000e+09,00:00:00
3,562,05q4,2005-12-31,2005-10-01,2006-01-18,5.168000e+09,5.168000e+09,00:00:00
4,562,06q1,2006-03-31,2006-01-01,2006-04-19,5.344000e+09,5.344000e+09,00:00:00
...,...,...,...,...,...,...,...,...
21232,1001,19q1,2019-03-31,2019-01-01,2019-05-09,8.558200e+07,8.558200e+07,16:03:00
21233,1001,19q2,2019-06-30,2019-04-01,2019-08-08,8.807500e+07,8.807500e+07,16:03:00
21234,1001,19q3,2019-09-30,2019-07-01,2019-11-07,9.055500e+07,9.055500e+07,16:03:00
21235,1001,19q4,2019-12-31,2019-10-01,2020-02-20,8.966800e+07,8.966800e+07,16:03:00


In [91]:
psql_reported_sales.to_csv('reported_sales.csv', index=False, header=False)

## migration of daily_sales_estimation

In [73]:
mysql_daily_sales_estimation = mysql_db.get_daily_estimation_pandas()
mysql_daily_sales_estimation = mysql_daily_sales_estimation.drop(columns={'id_daily_sales_estimation'})
mysql_daily_sales_estimation

,company_id,quarter_group,year_estimation,quarter_estimation,datetime,estimation
0,1,05q1,2005,1,2001-01-01,0.000000e+00
1,1,05q2,2005,2,2001-01-01,0.000000e+00
2,1,05q3,2005,3,2001-01-01,0.000000e+00
3,1,05q4,2005,4,2005-12-13,5.182000e+09
4,1,05q4,2005,4,2006-01-13,5.271000e+09
...,...,...,...,...,...,...
1186170,534,20q2,2020,2,2020-04-28,8.298300e+07
1186171,534,20q2,2020,2,2020-05-01,7.948300e+07
1186172,534,20q2,2020,2,2020-05-06,7.692500e+07
1186173,534,20q2,2020,2,2020-05-08,6.977500e+07


In [85]:
# change the company_id of the table
psql_daily_sales_estimation = mysql_daily_sales_estimation.copy()
psql_daily_sales_estimation['company_id'] = psql_daily_sales_estimation['company_id'].map(lambda x: change_company_id(psql_company, mysql_company, x), list(psql_daily_sales_estimation['company_id']))

psql_daily_sales_estimation

,company_id,quarter_group,end_quarter_date,start_quarter_date,filing_date,actual_sales,sales_reported,time_announcement
0,562,05q1,2005-03-31,2004-12-31,2005-04-20,4.750000e+09,4.750000e+09,00:00:00
1,562,05q2,2005-06-30,2005-04-01,2005-07-20,5.309000e+09,5.309000e+09,00:00:00
2,562,05q3,2005-09-30,2005-07-01,2005-10-19,5.485000e+09,5.485000e+09,00:00:00
3,562,05q4,2005-12-31,2005-10-01,2006-01-18,5.168000e+09,5.168000e+09,00:00:00
4,562,06q1,2006-03-31,2006-01-01,2006-04-19,5.344000e+09,5.344000e+09,00:00:00
...,...,...,...,...,...,...,...,...
21232,1001,19q1,2019-03-31,2019-01-01,2019-05-09,8.558200e+07,8.558200e+07,16:03:00
21233,1001,19q2,2019-06-30,2019-04-01,2019-08-08,8.807500e+07,8.807500e+07,16:03:00
21234,1001,19q3,2019-09-30,2019-07-01,2019-11-07,9.055500e+07,9.055500e+07,16:03:00
21235,1001,19q4,2019-12-31,2019-10-01,2020-02-20,8.966800e+07,8.966800e+07,16:03:00


In [90]:
psql_daily_sales_estimation.to_csv('daily_sales_estimation.csv', index=False, header=False)

## migration of historical_price

In [89]:
mysql_historical_price = mysql_db.get_historical_prices_pandas()
mysql_historical_price

,company_id,date,close,high,low,open
0,1,2013-12-09,24.60,25.440,23.4501,23.95
1,1,2013-12-10,24.88,25.170,24.4100,24.54
2,1,2013-12-11,25.99,27.200,25.3700,25.44
3,1,2013-12-12,25.45,26.710,25.4500,26.20
4,1,2013-12-13,26.23,26.300,25.5201,25.61
...,...,...,...,...,...,...
1441557,534,2020-05-20,2.64,2.760,2.6000,2.67
1441558,534,2020-05-21,2.58,2.655,2.5350,2.63
1441559,534,2020-05-22,2.55,2.610,2.4600,2.61
1441560,534,2020-05-26,2.63,2.665,2.5500,2.60


In [92]:
# change the company_id of the table
psql_historical_price = mysql_historical_price.copy()
psql_historical_price['company_id'] = psql_historical_price['company_id'].map(lambda x: change_company_id(psql_company, mysql_company, x), list(psql_historical_price['company_id']))

psql_historical_price

,company_id,date,close,high,low,open
0,562,2013-12-09,24.60,25.440,23.4501,23.95
1,562,2013-12-10,24.88,25.170,24.4100,24.54
2,562,2013-12-11,25.99,27.200,25.3700,25.44
3,562,2013-12-12,25.45,26.710,25.4500,26.20
4,562,2013-12-13,26.23,26.300,25.5201,25.61
...,...,...,...,...,...,...
1441557,1001,2020-05-20,2.64,2.760,2.6000,2.67
1441558,1001,2020-05-21,2.58,2.655,2.5350,2.63
1441559,1001,2020-05-22,2.55,2.610,2.4600,2.61
1441560,1001,2020-05-26,2.63,2.665,2.5500,2.60


In [93]:
psql_historical_price.to_csv('historical_price.csv', index=False, header=False)